In [1]:
# now attempt loading ecog data.

import mne
import numpy as np
import pandas as pd
import scipy
import re

# some hyperparameter type things :)
sr = 1000
t = 1000
affix = '_raw'

In [3]:
# load the raw data as a .set file
# do preprocessing if necessary (can tweak this part)

fileList = []
subjList = ['bp', 'cc', 'ht', 'jp', 'mv', 'wc', 'zt'] #  'jc' 'wm', <take out

for subj in subjList:
    data = scipy.io.loadmat(f'preprocessing/raw_data/{subj}_fingerflex.mat')['data'].T
    ch = data.shape[0]
    # print(data.shape)
    events = scipy.io.loadmat(f'preprocessing/{subj}_labels.mat')['Labels'][:, 1:3]

    events = np.insert(events, 1, np.zeros(len(events)), axis=1)
    
    features = []
    labels = []

    for event in events:
        time = int(event[0])
        # if sr == 1000: time = int(time/5)
        type = int(event[2])
        features.append([data[i][time:time+1000] for i in range(ch)])
        labels.append(type - 1)
    features = np.array(features)
    labels = np.array(labels)

    # print(features.shape, labels.shape)
    fileList.append((subj, np.array(features), np.array(labels)))


for subj, file, labels in fileList:
    np.save(f'pickles/{subj}_features{affix}', file)
    np.save(f'pickles/{subj}_labels{affix}', labels)

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessing/bp_labels.mat'

In [2]:
# load .npy files
fileList = []

for subj in ['bp', 'cc', 'ht', 'jp', 'mv', 'wc', 'zt']:
    features = np.load(f'pickles/{subj}_features{affix}.npy')
    labels = np.load(f'pickles/{subj}_labels{affix}.npy')
    fileList.append((subj, features, labels))

FileNotFoundError: [Errno 2] No such file or directory: 'pickles/bp_features_raw.npy'

In [ ]:
#2D CNN w/ Adham's recommendations

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_2D(nn.Module):
    def __init__(self):
        super(ConvNet_2D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            # nn.Flatten(),
            # lol ok to calculate the size of the linear layer ...
            # width = starting width (time) - combined (kernel sizes + 1) < constant = 1000 - 4 994
            # height = starting height (channels) - combined (kernel heights + 1) < variable = ch - 12
            # nn.Linear(() * 994 * 32, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        x_flat = x.shape[1] * x.shape[2] * x.shape[3]
        flat = nn.Flatten()
        x = flat(x)
        linear = nn.Linear(x_flat, 256)
        # print(x.shape, x_flat)
        x = linear(x)
        x = self.hidden(x)
        return x


In [ ]:
# run CNN
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
accuracies = []

for subj, features, labels in fileList:
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_2D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    # Train the neural network
    for epoch in range(50):
        # print("epoch = ",epoch)
        conv_net.train()
        data = X_train
        targets = y_train
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

    # Evaluate the neural network
    conv_net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        data = X_test
        targets = y_test
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)
        conv_net_predictions = conv_net(data)
        _, predicted = torch.max(conv_net_predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    print(f"test accuracy {subj}= ",correct/total)
    accuracies.append(correct/total)

print(f'avg accuracy = {np.mean(accuracies)}')